In [84]:
%matplotlib inline

import datacube
import numpy as np
import pandas as pd
from odc.ui import DcViewer
from odc.ui import with_ui_cbk
import rasterio.crs
import geopandas as gpd
import matplotlib.pyplot as plt
from datacube.utils.geometry import Geometry, CRS
from datacube.utils import geometry

import sys
sys.path.insert(1, '../Tools/')
from dea_tools.datahandling import load_ard
from dea_tools.bandindices import calculate_indices
from dea_tools.plotting import rgb, map_shapefile
from dea_tools.temporal import time_buffer
from dea_tools.spatial import xr_rasterize
from datacube.utils import masking
import matplotlib.pyplot as plt
from dea_tools.datahandling import wofs_fuser
from dea_tools.plotting import plot_wo
from dea_tools.dask import create_local_dask_cluster


dc = datacube.Datacube(app="02_explore_surface_water_data")

# Create a csv file to save results
with open('water_results.csv', 'w') as f:
    f.write('name,date,wet_pixels,dry_pixels,nodata_pixels,total_pixels\n')

# Open shapefile
vector_file = 'TwoTinyPolygons.shp'
attribute_col = 'Name'
gdf = gpd.read_file(vector_file)

# Loop through polygons and extract data
for index, row in gdf.iterrows():
   
    print(f'Feature: {index + 1}/{len(gdf)}')
   
    # Get site ID
    ID = str(row[attribute_col])
   
    # Extract the feature's geometry as a datacube geometry object
    geom = geometry.Geometry(geom=row.geometry, crs=gdf.crs)
   
    time_to_drill = ('2021-09', '2021-11')

    query = {'geopolygon': geom,
             'time': time_to_drill}

    # Load data for our polygon and time period
    wo = dc.load(product='ga_ls_wo_3',
                 output_crs='EPSG:3577',
                 resolution=(-30, 30),
                 group_by='solar_day',
                 fuse_func=wofs_fuser,
                 **query)
    
    # Create binary arrays
    wo_wet = masking.make_mask(wo, wet=True)
    wo_cloud = masking.make_mask(wo, cloud=True)
    wo_shadow = masking.make_mask(wo, cloud_shadow=True)
    wo_nodata = masking.make_mask(wo, nodata=True)
    
    # Create classified array where nodata = 0, dry = 1, and wet = 2
    wet_dry = wo_wet.astype(np.uint8)
    wet_dry.water.values[(wo_wet.water.values == 1) & (wo_cloud.water.values == 0) & (wo_shadow.water.values == 0) & (wo_nodata.water.values == 0)] = 2
    wet_dry.water.values[(wo_wet.water.values == 0) & (wo_cloud.water.values == 0) & (wo_shadow.water.values == 0) & (wo_nodata.water.values == 0)] = 1
    
    # Resample to combine each 1 month of data into a composite
    monthly_wet_dry = wet_dry.resample(time='1m').max()
    
    # Create binary wet, dry and nodata arrays
    monthly_wet = monthly_wet_dry.where(monthly_wet_dry.water.values == 2)
    monthly_dry = monthly_wet_dry.where(monthly_wet_dry.water.values == 1)
    monthly_nodata = monthly_wet_dry.where(monthly_wet_dry.water.values == 0)
    monthly_wet.water.values[monthly_wet.water.values == 2] = 1
    monthly_dry.water.values[monthly_dry.water.values == 1] = 1
    monthly_nodata.water.values[monthly_nodata.water.values == 0] = 1
    
    # Mask outside of polygon
    poly_mask = xr_rasterize(gdf.iloc[[index]], wo)
    monthly_wet = monthly_wet.where(poly_mask)
    monthly_dry = monthly_dry.where(poly_mask)
    monthly_nodata = monthly_nodata.where(poly_mask)
    
    # Calculate the sum of wet and dry pixels for each month
    wet_pixels = monthly_wet.sum(dim=['x', 'y'])
    dry_pixels = monthly_dry.sum(dim=['x', 'y'])
    nodata_pixels = monthly_nodata.sum(dim=['x', 'y'])
    
    # Write sum for wet, dry and nodata for each time period, to the CSV file
    for d in wet_pixels["time"].values:
        wet_d = wet_pixels["water"][wet_pixels["time"] == d]
        dry_d = dry_pixels["water"][dry_pixels["time"] == d]
        nodata_d = nodata_pixels["water"][nodata_pixels["time"] == d]
        total_d = wet_d + dry_d + nodata_d
        with open('water_results.csv', 'a') as f:
            f.write('%s,%s,%i,%i,%i,%i\n'%(ID, d, wet_d, dry_d, nodata_d, total_d))

print("Completed")

Feature: 1/2
Feature: 2/2
Completed
